In [ ]:
!pip install transformers text-hammer pyreadstat

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup , AutoTokenizer, TFAutoModel
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, f1_score
import matplotlib.pyplot as plt
import os
import io
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold
from sklearn import svm
from collections import defaultdict
import text_hammer as th
import pandas as pd
import tensorflow as tf , keras
import transformers
import random as rd
import keras.backend as K
from numpy.random import seed
from tensorflow.keras import layers
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# identify and specify the GPU as the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
def compute_validation_loss(model, validation_dataloader):
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in validation_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    average_val_loss = total_val_loss / len(validation_dataloader)
    return average_val_loss

def metric2(y_true, y_pred_classes):
    n = y_true.size(0)
    # Calculate errors where prediction is off by 1 class
    res = torch.abs(y_true - y_pred_classes)
    count_error = torch.sum(res == 1, dtype=torch.float32)
    metric = 1 - count_error / n
    return metric.item()

def metric2_2(y_true, y_pred):
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # Calculate the number of predictions off by 1 class
    off_by_one = np.sum(np.abs(y_true - y_pred) == 1)
    # Calculate the metric
    metric = 1 - off_by_one / len(y_true)
    return metric


In [ ]:
def get_clean(x):
    mention = r'@\w+'
    hash = r'#\w+'
    x = str(x).lower().replace('\\', '').replace('_', '')
    x = re.sub(r'[^\x00-\x7F]+', ' ', x)
    x = th.cont_exp(x)
    x = th.remove_emails(x)
    x = th.remove_urls(x)
    x = re.sub(mention, ' ', x)
    x = re.sub(hash, ' ', x)
    x = th.remove_html_tags(x)
    x = th.remove_rt(x)
    x = th.remove_accented_chars(x)
    x = th.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    x = re.sub(r'\s+', ' ', x).strip()
    x = re.sub(r'\w*\d+\w*', ' ', x).strip()
    return x

In [ ]:
df = pd.read_spss("/content/drive/MyDrive/VA_EN_TU_2012-2020_3000_tweets_relevant_V03_labeled_1200_cleaned.sav")
# Original labels: 0 - positive, 1 - negative, 2 - neutral
# Remapping dictionary to align with RoBERTa's expected labels
label_mapping = {1: 2, 2: 0, 3: 1}

# Remapped labels: 0 - negative, 1 - neutral, 2 - positive
df['Label_B_emotion'] = df['Label_B_emotion'].replace(label_mapping)
df['Label_B_emotion'] = df['Label_B_emotion'].astype(int)
dff = df[['text','Label_B_emotion']].copy()
dff['Label_B_emotion'].unique()

In [ ]:
dff['cleaned_data'] = dff['text'].apply(get_clean)

In [ ]:
X = dff['text'].tolist()
y = dff['Label_B_emotion'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# n-grams
n_gram_ranges = [(1, 1), (1, 2), (1, 3)]
val_predictions = defaultdict(list)
#test_predictions = []
test_predictions = {}
val_accuracies = {}

for n_gram_range in n_gram_ranges:
    print(f"--------------------------- {n_gram_range} ---------------------------")
    # TF-IDF vectorizer with different n-grams
    vectorizer = TfidfVectorizer(ngram_range=n_gram_range, max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    #svm_model = SVC(kernel='linear')

    val_scores = []
    for train_index, val_index in kfold.split(X_train_tfidf):
        X_kf_train, X_kf_val = X_train_tfidf[train_index], X_train_tfidf[val_index]
        y_kf_train, y_kf_val = np.array(y_train)[train_index], np.array(y_train)[val_index]

        svm_model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', random_state=42)
        svm_model.fit(X_kf_train, y_kf_train)
        y_val_pred = svm_model.predict(X_kf_val)
        val_scores.append(accuracy_score(y_kf_val, y_val_pred))
        #val_predictions[n_gram_range].append(y_val_pred)

    avg_val_score = np.mean(val_scores)
    val_accuracies[n_gram_range] = avg_val_score
    print(f"Average validation accuracy for n-grams {n_gram_range}: {avg_val_score}")

    svm_model.fit(X_train_tfidf, np.array(y_train))
    X_test_tfidf = vectorizer.transform(X_test)
    y_test_pred = svm_model.predict(X_test_tfidf)
    test_predictions[n_gram_range] = y_test_pred

    test_score = accuracy_score(y_test, y_test_pred)
    test_metric2 = metric2_2(y_test, y_test_pred)

    print(f"Test accuracy for n-grams {n_gram_range}: {test_score}")
    print(f"Test metric 2 for n-grams {n_gram_range}: {test_metric2}")
    print("Classification Report: \n", classification_report(y_test, y_test_pred,target_names=['Negative','Neutral','Positive']))

    cm = confusion_matrix(y_test, y_test_pred)
    target_names = ['negative','neutral','positive']
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_names,
                yticklabels=target_names)

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix [SVM]: Raw data')
    plt.show()

#### Cleaned Text

In [ ]:
X = dff['cleaned_data'].tolist()
y = dff['Label_B_emotion'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

n_gram_ranges = [(1, 1), (1, 2), (1, 3)]
val_predictions = defaultdict(list)
#test_predictions = []
test_predictions = {}
val_accuracies = {}

for n_gram_range in n_gram_ranges:
    print(f"--------------------------- {n_gram_range} ---------------------------")
    # TF-IDF vectorizer with different n-grams
    vectorizer = TfidfVectorizer(ngram_range=n_gram_range, max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    #svm_model = SVC(kernel='linear')

    val_scores = []
    for train_index, val_index in kfold.split(X_train_tfidf):
        X_kf_train, X_kf_val = X_train_tfidf[train_index], X_train_tfidf[val_index]
        y_kf_train, y_kf_val = np.array(y_train)[train_index], np.array(y_train)[val_index]

        svm_model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', random_state=42)
        svm_model.fit(X_kf_train, y_kf_train)
        y_val_pred = svm_model.predict(X_kf_val)
        val_scores.append(accuracy_score(y_kf_val, y_val_pred))
        #val_predictions[n_gram_range].append(y_val_pred)

    avg_val_score = np.mean(val_scores)
    val_accuracies[n_gram_range] = avg_val_score
    print(f"Average validation accuracy for n-grams {n_gram_range}: {avg_val_score}")

    svm_model.fit(X_train_tfidf, np.array(y_train))
    X_test_tfidf = vectorizer.transform(X_test)
    y_test_pred = svm_model.predict(X_test_tfidf)
    test_predictions[n_gram_range] = y_test_pred

    test_score = accuracy_score(y_test, y_test_pred)
    test_metric2 = metric2_2(y_test, y_test_pred)

    print(f"Test accuracy for n-grams {n_gram_range}: {test_score}")
    print(f"Test metric 2 for n-grams {n_gram_range}: {test_metric2}")
    print("Classification Report: \n", classification_report(y_test, y_test_pred,target_names=['Negative','Neutral','Positive']))

    cm = confusion_matrix(y_test, y_test_pred)
    target_names = ['negative','neutral','positive']
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_names,
                yticklabels=target_names)

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix [SVM]: Cleaned data')
    plt.show()